<h1 align="center">Spatial Data with Python Workshop</h1>

<h4 align="center"><i>Created by <a href="https://github.com/amitparikh1">Amit Parikh</a> for Carolina Analytics and Data Science on 3/1/2023</i></h4>

 
**Spatial Data** can be defined as any series of data that has an implicit or explicit association with a geographic location. Spatial data is all around us and is constantly being collected. 

In this workshop we will go over some common examples of spatial datasets, data types that are specific to spatial data, how to best analyze spatial data in Python using GeoPandas, and explore ways we can visualize spatial data. 

This workshop is meant to be **introductory** and is open to all skill levels. A basic understanding of Python and Pandas would be helpful, but not necessary. To familiarize yourself with Python and Pandas, check out our <a href="https://youtube.com/playlist?list=PLtOYSqZWWG7L1tUaEgUwtJP0d9xYHN7o1"> CADS intro YouTube series </a>. 

## Examples of Spatial Data

## Data Types Specific to Spatial Data <a href="https://www.safe.com/what-is/spatial-data/">(Source)</a>

### Vector

Vector data is the most commmon type of spatial data. Data in this form comes in three types: **point**, **lines**, and **polygons**.

<img src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png" width =250/>

**Points** are represented by a ***(latitude, longitude)*** coordinate pair that corresponds to a physical location in the world. 

    Example: the precise location of a thermometer recording daily temperatures.

**Lines** are represented by ***2+*** points that are ***connected together***. Each "bend" in a line is represented by a new point. 

    Example: a line can be used to mark a GPS route from point A to point B. 
    
**Polygons** are represented by ***3+*** points that are ***connected together*** and ***closed***. 

    Example: a polygon can mark the boundary of a county, zipcode, state, or country. 
<img src="https://digital.newberry.org/ahcb/images/statepages/North_Carolina.gif" width =250/>

### Raster

Raster data is gridded data where each pixel is associated with a specific geographical location.

    Example: commonly used to represent satellite image data. 
    
<img src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-raster/raster_concept.png" width =250/>

### Attributes

Attributes refer to additional data that describes a geographical feature. 

    Example: A polygon that represents a home has attributes such as # of bedrooms, # of bathrooms, square footage etc. 

### Geocoding

The process by which tabular data such as address is analyzed and converted into latitude, longitude coordinates. 

    Example: Say a Real Estate investor had the street addresses of 100 homes in his portfolio! To visualize where these homes are on a map, you would first geocode the addresses to lat, lng coordinates. 
    
Many API services offer free tiers for geocoding such as <a href="https://developers.google.com/maps">Google Maps </a> and <a href= "https://developer.here.com/"> HERE </a>.

## Using GeoPandas to Analyze Spatial Data

## Visualizing Spatial Data